In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
tf.enable_eager_execution()

In [3]:
import matplotlib.pyplot as plt 

In [4]:
import os

In [5]:
def cnn_model(features,labels,mode):
    input_layer = tf.reshape(features["x"],[-1,28,28,1])
    
    conv1 = tf.layers.Conv2D(inputs = input_layer,
                             filters = 32,
                             kernal_size =[5,5],
                             padding = 'same',
                             activation= tf.nn.relu
                             )
    pool1 = tf.layers.max_pooling2d(inputs=conv1,strides=2,pool_size=[2,2])
    
    conv2 = tf.layers.Conv2D(inputs = pool1,
                             filters = 64,
                             kernal_size =[5,5],
                             padding = 'same',
                             activation= tf.nn.relu
                             )
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
    
    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
    
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    logits = tf.layers.dense(inputs=dropout, units=10)
    
    predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
         mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

    